# Import Modules

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np 
import datetime as dt 

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import seaborn as sns 
import plotly.express as px
import plotly.graph_objects as go 
from mpl_toolkits.mplot3d import Axes3D
from yellowbrick.cluster import KElbowVisualizer

# Read the file

In [2]:
df = pd.read_csv('data.csv', header=0, encoding='unicode_escape',dtype={'CustomerID' : str}, parse_dates=['InvoiceDate'], infer_datetime_format=True)
display(df.head(10))
print(len(df))

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047,United Kingdom


541909


## Make a summary

In [3]:

def summary(df, num_rows=3):

    """
    Generate a summary of the DataFrame.

    Parameters:
    - df: Dataframe
    - num_rows: Number of rows to display

    Returns:
    - DataFrame containing summury
    """

    #numeric_collumns = df.select_dtypes(include=['number']).columns

    # Basic information
    print(f'Data shape: {df.shape}')

    # Data types, missing values, and unique values
    data_info = pd.DataFrame({
        'Data Type': df.dtypes,
        'Missing': df.isnull().sum(),
        'Percent Missing': df.isnull().mean() * 100,
        'Unique Values': df.nunique()
    })

    # Descriptive statistics
    desc_stats = df.describe(include='all').transpose()

    #  # Extract n values for each row
    row_values = pd.DataFrame()
    for i in range(num_rows):
        row_values[f'Value-{i + 1}'] = df.iloc[i]
    
    # Combine the information

    summary_df = pd.concat([data_info, desc_stats[['min', 'max']], row_values], axis=1)
   
    return summary_df

df_summary = summary(df)
display(df_summary)

# Quantity with missing values
quantity = pd.DataFrame([{'Products' : len(df['StockCode'].value_counts()), 'Customers' : (df['CustomerID'].nunique())}],
             columns=['Products', 'Customers'], index=['quantity']) # Names in columns should match
print(quantity)

Data shape: (541909, 8)


,Data Type,Missing,Percent Missing,Unique Values,min,max,Value-1,Value-2,Value-3
InvoiceNo,object,0,0.000000,25900,NaN,NaN,536365,536365,536365
StockCode,object,0,0.000000,4070,NaN,NaN,85123A,71053,84406B
Description,object,1454,0.268311,4223,NaN,NaN,WHITE HANGING HEART T-LIGHT HOLDER,WHITE METAL LANTERN,CREAM CUPID HEARTS COAT HANGER
Quantity,int64,0,0.000000,722,-80995.0,80995.0,6,6,8
InvoiceDate,datetime64[ns],0,0.000000,23260,2010-12-01 08:26:00,2011-12-09 12:50:00,2010-12-01 08:26:00,2010-12-01 08:26:00,2010-12-01 08:26:00
UnitPrice,float64,0,0.000000,1630,-11062.06,38970.0,2.55,3.39,2.75
CustomerID,object,135080,24.926694,4372,NaN,NaN,17850,17850,17850
Country,object,0,0.000000,38,NaN,NaN,United Kingdom,United Kingdom,United Kingdom


          Products  Customers
quantity      4070       4372


### Table analys (rename later)

In [15]:
def replace_with_tresholds(Dataframe, variable, q1 = 0.25, q3 = 0.75):
    df_copy = Dataframe.copy()
    quartile1, quartile3 = df_copy[variable].quantile([q1, q3])
    iqr = quartile3 - quartile1

    lower_limit = quartile3 + 1.5 * iqr
    upper_limit = quartile3 - 1.5 * iqr

    df_copy[variable] = df_copy[variable].clip(lower=lower_limit, upper=upper_limit )

    return df_copy

def preprocess(DataFrame):
    df = DataFrame.copy()

    # Missing values
    print(df.isnull().sum())

    # Remove missing values
    df = df.dropna()

    # Cancelled Orders & Quantity
    print(df[~df['InvoiceNo'].str.contains('C', na=False)].head(2))
    df = df[~df['InvoiceNo'].str.contains('C', na=False)]

    df = df[df['Quantity'] > 0]

    # Replacing Outliers
    variables_for_process = ['Quantity', 'UnitPrice']
    for variable in variables_for_process:
        df = replace_with_tresholds(df, variable, q1=0.01, q3=0.99)

    # Total Price
    df['TotalPrice'] = df['Quantity'] * df['UnitPrice'] # New column
    
    return df

df_preprocessed = preprocess(df)
df_preprocessed


InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64
  InvoiceNo StockCode                         Description  Quantity  \
0    536365    85123A  WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                 WHITE METAL LANTERN         6   

          InvoiceDate  UnitPrice CustomerID         Country  
0 2010-12-01 08:26:00       2.55      17850  United Kingdom  
1 2010-12-01 08:26:00       3.39      17850  United Kingdom  


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6.0,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12.0,2011-12-09 12:50:00,0.85,12680,France,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6.0,2011-12-09 12:50:00,2.10,12680,France,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4.0,2011-12-09 12:50:00,4.15,12680,France,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4.0,2011-12-09 12:50:00,4.15,12680,France,16.60


## EDA

In [16]:
purchase = df_preprocessed.groupby(['Country'], as_index=False)